In [1]:
!pip install tensorflow==2.2.0

In [2]:
!pip install bert4keras

In [3]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
# Change work directory and check it
import os
os.chdir('/content/drive/My Drive/')
!ls

chinese_L-12_H-768_A-12  model.ipynb  model.pth      Untitled
dataset			 model.pt     pytorch.ipynb  Untitled（副本）


In [5]:
import os
# import pandas as pd
# import numpy as np
# import math
# import tensorflow as tf

# from bert4keras.backend import keras, set_gelu   # Warning: Using tensorflow backend
# from bert4keras.tokenizers import Tokenizer
# from bert4keras.models import build_transformer_model
# from bert4keras.snippets import to_array
# from bert4keras.snippets import sequence_padding, DataGenerator
# from bert4keras.snippets import open
# from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr

# from keras.layers.core import *  # 包括Lambda, Dropout, Dense
# from keras.models import Model
# import keras.backend as K

import numpy as np
from bert4keras.backend import keras, set_gelu
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam, extend_with_piecewise_linear_lr
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Lambda, Dense

Using TensorFlow backend.


In [6]:
# set model path and hyper parameters
set_gelu('tanh')  # 切换gelu版本

num_classes = 2
maxlen = 256
dropout = 0.5
batch_size = 16
epochs = 3
lr = 2e-5
pretrain_model_path = 'chinese_L-12_H-768_A-12'
config_path = os.path.join(pretrain_model_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrain_model_path, 'bert_model.ckpt')
dict_path = os.path.join(pretrain_model_path, 'vocab.txt')

In [7]:
# Load data
def LoadData(loadpath):
    train = np.load(os.path.join(loadpath, 'train.npy'))
    valid = np.load(os.path.join(loadpath, 'valid.npy'))
    test = np.load(os.path.join(loadpath, 'test.npy'))
    # train_data = np.delete(train_data, 1, axis=1)
    # valid_data = np.delete(valid_data, 1, axis=1)
    # test_data = np.delete(test_data, 1, axis=1)
    # train_df = pd.DataFrame(train_data)
    # train_df[1] = train_df[1].astype(np.int32)
    # valid_df = pd.DataFrame(valid_data)
    # valid_df[1] = valid_df[1].astype(np.int32)
    # test_df = pd.DataFrame(test_data)
    # test_df[1] = test_df[1].astype(np.int32)
    # train_data = train_df.to_numpy()
    # valid_data = valid_df.to_numpy()
    # test_data = test_df.to_numpy()
    train_data = []
    for i in range(len(train)):
        train_data.append((train[i][0], int(train[i][2])))
    valid_data = []
    for i in range(len(valid)):
      valid_data.append((valid[i][0], int(valid[i][2])))
    test_data=[]
    for i in range(len(test)):
        test_data.append((test[i][0], int(test[i][2])))
    return train_data, valid_data, test_data

loadpath = "dataset/"
train_data, valid_data, test_data = LoadData(loadpath)
print(len(train_data))
print(train_data[0])
print(len(valid_data))
print(valid_data[0])
print(len(test_data))
print(test_data[0])

27240
('【男子用石块砸袋鼠称;难得来一趟动物园;】3月26日，扬州茱萸湾动物园内，一名男子疑因嫌弃袋鼠躺卧在地上画面不够精彩，多次用砸石头砸向袋鼠并进行拍摄。旁边游客阻止他时，男子竟然辩称“难得来一趟”（交汇点）袋鼠：我一通电话从澳洲叫一百个亲戚来锤你胸口。...', 0)
3406
('#2018年春运交通安全提示#一年一度的春运来临，这一期间人流、物流、车流大幅增长，返乡、探亲、旅行、就学等各种客流互相叠加，交通出行面临很多困难和风险。保障出行安全首先要牢固树立安全意识、规则意识和文明意识，从源头化解安全风险。泰安交警提示您：1.乘坐客运车辆出行时请选择正规的客运车辆，并在站内上车，勿乘坐站外私揽客源的客车，勿贪便宜方便乘坐黑包车或超员车。2.春节假期期间，拼车、租车、自驾车出游的朋友，请勿超员，请广大交通参与者拒绝乘座超员车辆。3.春运期间公路交通流量大。出行前要通过电视、广播等方式掌握路况信息，了解天气状况和道路通行信息，合理安排行程和出行路线。出行前，请确保车辆性能良好，状态安全；长途驾驶过程中，请保持注意力集中，时刻警惕事故发生。4.春节自驾车回家过年，请合理安排行程，出发前保证充足睡眠，注意劳逸结合，尽量避开习惯睡眠时间行车，全天累计驾驶时间不超过8小时，连续驾驶4小时要选择安全地点停车休息20分钟以上。5.春运期间举家返乡、节日出游增多，请您系好安全带的同时，莫忘为儿童选用适合其年龄、体重的儿童安全座椅，切莫怀抱儿童或让儿童乘坐站立在副驾驶位置。6.春节期间，亲朋好友合家团圆，聚餐饮酒增多。饮酒会导致人反应迟钝、行动迟缓，驾驶人控制车辆能力下降。为了您和他人的安全，请一定记得互相提醒：喝酒不开车，开车不喝酒！7.春节期间，走亲访友聚会多，可能因此而休息不足，引发疲劳。疲劳导致人的判断能力下降、反应迟钝，增加操作失误。疲劳时请不要勉强驾车，应及时休息，待疲劳减轻后再驾驶车辆。8.春节前后，农村地区赶集购物、走亲访友活动频繁，交通出行增多。拖拉机、货车货厢违法载人以及客货混装隐患多、危害大。请农民朋友不要乘坐货车、拖拉机、超员车出行，切勿人货混装。9.微型面包车安全性能较差，特别是农村道路狭窄，陡坡、急弯较多，易失控发生事故。春节期间亲朋好友出行，切勿驾驶及乘坐超员超速面包车。10.三轮汽车、低速货车安全技术性能不高、车体结构安全防护性差，请不要

In [8]:
# 建立分词器
# 【加载词典】
def LoadDict(dict_path):
    token_dict = {}
    with open(dict_path, 'r', encoding='utf-8') as f:
        vocab = f.read().split("\n")
        while '' in vocab:
            vocab.remove('')
        for i in range(0, len(vocab)):
            token = vocab[i]
            token_dict[token] = i
    return token_dict


# 【定义分词器】
class Task3Tokenizer(Tokenizer):
    def _tokenize(self, text):
        tokenized_text = []
        for c in text:
            if c in self._token_dict:
                tokenized_text.append(c)
            elif self._is_space(c):
                tokenized_text.append('[unused1]')
            else:
                tokenized_text.append('[UNK]')    # 如果token不在字典内，则用[UKN]代替，UKN=unkown
        return tokenized_text
token_dict = LoadDict(dict_path)
tokenizer = Task3Tokenizer(token_dict)

In [9]:
# 【序列padding】
def seq_padding(X, padding=0):
    length = [len(x) for x in X]
    max_length = max(length)
    return np.array([
        np.concatenate([x, [padding] * (max_length - len(x))]) if len(x) < max_length else x for x in X
    ])

# 【定义生成器】
# 修改成了可以处理句子对的模式
# class data_generator:
#     def __init__(self, data, batch_size=batch_size):
#         self.data = data
#         self.batch_size = batch_size
#         self.steps = len(self.data) // self.batch_size
#         if len(self.data) % self.batch_size != 0:
#             self.steps += 1
#     def __len__(self):
#         return self.steps
#     def __iter__(self):
#         while True:
#             idxs = list(range(len(self.data)))
#             np.random.shuffle(idxs)
#             X1, X2, Y = [], [], []
#             for i in idxs:
#                 d = self.data[i]
#                 text_a = d[0][:maxlen]
#                 text_b = d[1][:maxlen]
#                 y = int(d[2])
#                 x1, x2 = tokenizer.encode(first_text=text_a, second_text=text_b, maxlen=maxlen)
#                 X1.append(x1)
#                 X2.append(x2)
#                 Y.append([y])
#                 if len(X1) == self.batch_size or i == idxs[-1]:
#                     X1 = seq_padding(X1)
#                     X2 = seq_padding(X2)
#                     Y = seq_padding(Y)
#                     yield [X1, X2], Y
#                     [X1, X2, Y] = [], [], []

class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                # batch_token_ids = sequence_padding(batch_token_ids)
                # batch_segment_ids = sequence_padding(batch_segment_ids)
                # batch_labels = sequence_padding(batch_labels)
                batch_token_ids = seq_padding(batch_token_ids)
                batch_segment_ids = seq_padding(batch_segment_ids)
                batch_labels = seq_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []

# 转换数据集
train_generator = data_generator(train_data, batch_size)
valid_generator = data_generator(valid_data, batch_size)
test_generator = data_generator(test_data, batch_size)

In [10]:
# Load model
# model = build_transformer_model(config_path=config_path,
#                                 checkpoint_path=checkpoint_path)
# output = Lambda(lambda x: x[:, 0], name='CLS-token')(model.output)
# output = Dense(units=num_classes, activation='softmax', kernel_initializer='glorot_uniform')(output)
# model = Model(model.input, output)
# model.compile(
#     loss='binary_crossentropy',
#     optimizer=Adam(lr),
#     metrics=['accuracy'])
# model.summary()

# 派生为带分段线性学习率的优化器。
# 其中name参数可选，但最好填入，以区分不同的派生优化器。
model = build_transformer_model(config_path=config_path,
                                checkpoint_path=checkpoint_path,
                                return_keras_model=False)
output = Lambda(lambda x: x[:, 0], name='CLS-token')(model.output) # 取[CLS]作为分类依据
output = Dense(units=num_classes, activation='softmax', kernel_initializer='glorot_uniform')(output) # bert的initializer会造成loss=nan
model = keras.models.Model(model.input, output)
AdamLR = extend_with_piecewise_linear_lr(Adam, name='AdamLR')
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=AdamLR(learning_rate=lr, lr_schedule={
        1000: 1,
        2000: 0.1
    }),
    metrics=['accuracy'],
)

# model.summary()

In [11]:
def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total


class Evaluator(keras.callbacks.Callback):
    """评估与保存
    """
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(valid_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('best_model.weights')
        test_acc = evaluate(test_generator)
        print(
            u'val_acc: %.5f, best_val_acc: %.5f, test_acc: %.5f\n' %
            (val_acc, self.best_val_acc, test_acc)
        )

evaluator = Evaluator()

In [ ]:
model.fit_generator(
    train_generator.__iter__(),
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    # validation_data=valid_generator.__iter__(),
    # validation_steps=len(valid_generator),
    callbacks=[evaluator],
    verbose=1
)

model.load_weights('best_model.weights')
model.save("my_model")
print(u'final test acc: %05f\n' % (evaluate(test_generator)))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/3
 553/1703 [========>.....................] - ETA: 34:55 - loss: 0.2841 - accuracy: 0.8648